In [11]:
import pandas
dir = "/mnt/c/Users/Theo/Desktop/OneDrive_2022-12-14/AGILE CST-2 mutation outputs"
meta_file = "PRJNA854613_sample-accession-numbers.tsv"
meta_df = pandas.read_csv(f"{meta_file}", sep="\t")

In [3]:

df = pandas.read_csv(meta_file, sep="\t")

In [7]:
# iterate over all TSV files in the directory

dfs = {}
import os, tqdm
for file in tqdm.tqdm(os.listdir(dir)):
    if file.endswith(".txt"):
        new_df = pandas.read_csv(os.path.join(dir, file), sep="\t")
        id, *_ = file.split("_")
        id = int(id)
        dfs[id] = new_df


100%|██████████| 374/374 [00:40<00:00,  9.16it/s]


In [13]:
meta_df['id'] = meta_df['BioSample.name'].str.removeprefix("AGL").astype(int)
# filter to id in keys of df
meta_df = meta_df[meta_df['id'].isin(dfs.keys())]

In [17]:
# sort by Title
meta_df = meta_df.sort_values('Title')
# split title into personID and day by _
meta_df['personID'], meta_df['day'] = meta_df['Title'].str.split('_', 1).str
grouped = meta_df.groupby('personID')

to_day_1 = {}

for name, group in grouped:
    d1 = group[group['day'] == 'd1']
    d1_id = d1['id'].values[0]
    for id in group['id']:
        if id != d1_id:
            to_day_1[id] = d1_id


<ipython-input-17-21b4b8bfc66d>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  meta_df['personID'], meta_df['day'] = meta_df['Title'].str.split('_', 1).str
<ipython-input-17-21b4b8bfc66d>:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  meta_df['personID'], meta_df['day'] = meta_df['Title'].str.split('_', 1).str


In [34]:
out_dir = "/mnt/d/MOV/out"
#os.mkdir(out_dir)
import numpy

for k, df in tqdm.tqdm(dfs.items()):
    if k not in to_day_1:
        continue
    d1_id = to_day_1[k]
    d1_df = dfs[d1_id].copy()
    #ref is the first character of the OrderOfNucs column
    d1_df['NewRef'] = d1_df['OrderOfNucs'].str[0]
    # if coverage is less than 20 then set NewRef to NaN
    d1_df.loc[d1_df['Coverage'] < 20, 'NewRef'] = numpy.nan

    # select only Position and NewRef
    d1_df = d1_df[['Position', 'NewRef']]
    # join to df
    df = df.merge(d1_df, on='Position', how='left')
    # write out
    df.to_csv(f"{out_dir}/{k}.tsv", sep="\t", index=False)



100%|██████████| 374/374 [01:38<00:00,  3.81it/s]


In [30]:
df

,Sample,Chr,Position,RefBase,Coverage,AvQual,Acnt,Apval,Ccnt,Cpval,...,NonRefCnt,CntTs,CntTv,OrderOfNucs,strandbias,ins_cnt,ins_mode,del_cnt,del_mode,Ncnt
0,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,1,A,5,0.000040,5.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A5:0;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
1,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,2,T,6,0.000033,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,T,A0:0;C0:0;T6:0;G0:0,NaN,NaN,NaN,NaN,0.0
2,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,3,T,8,0.000025,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,T,A0:0;C0:0;T8:0;G0:0,NaN,NaN,NaN,NaN,0.0
3,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,4,A,10,0.000020,10.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A9:1;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
4,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,5,A,20,0.000010,20.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A19:1;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29898,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,29899,A,33,0.002407,33.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A33:0;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
29899,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,29900,A,33,0.000006,33.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A33:0;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
29900,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,29901,A,33,0.000006,33.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A33:0;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
29901,999_CAAGCATACC-AAGGCGTCGC_L002.final.bam,NC_045512.2,29902,A,31,0.000006,31.0,0.0,0.0,1.0,...,0.0,0.0,0.0,A,A31:0;C0:0;T0:0;G0:0,NaN,NaN,NaN,NaN,0.0
